In [86]:
import pandas as pd
import re

Global variables


In [87]:
leagues_prefixs = ["SA1", "SA2L"]
clubs_list = []
champions_list = []

Transform functions


In [88]:
# Function to transform attendences DataFrame

def attendances_tranform():
    df_f = pd.read_csv("./RAW/ArabicSoccer/Attendances/Arabic_Attendances_2007_2023.csv")

    # In column "#" where the value is NaN is duplicate row
    # So I decide to remove using comparision "['#'] >= 1"
    df = df_f[df_f['#'] >= 1]
    
    # Reorganize and select columns
    df = df[["Stadium", "Capacity", "Spectators",
             "Average", "Matches", "sold out", "Club", "Year"]]

    # Pattern to separate the stadium name do club name
    pattern = "\B[ANH][Llab][-]?[a-zA-Z]+(?:-[a-zA-Z]+)?\s?[SFClub]{1,4}"

    # I use these if's because I can't catch these words in my pattern
    for i in range(len(df)):
        if "Al-Ansar" in df.iloc[i, 0]:
            df.iloc[i, 6] = "Al-Ansar"
            df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium"
        elif "Ohod Club" in df.iloc[i, 0]:
            df.iloc[i, 6] = "Ohod Club"
            df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium"
        elif "Damac FC" in df.iloc[i, 0]:
            df.iloc[i, 6] = "Damac FC"
            df.iloc[i, 0] = "Damac Club Stadium"
        else:
            # In re.findall() I use regex to catch teams name
            # King Abdullah Sports CityAl-Ittihad Club ----> ['Al-Ittihad Club']
            df.iloc[i, 6] = re.findall(pattern=pattern, string=df.iloc[i, 0])[0]

            # Using re.sub() I can replace the clubs names in String to "", leaving the line with just the name of the stadium
            df.iloc[i, 0] = re.sub(pattern=pattern, string=df.iloc[i, 0], repl="")

    # Reset the index, since there was a change in the DataFrame rows.
    df = df.reset_index()

    # Create a id column
    df["index"] = df.index + 1
    df = df.rename(columns={"index": "id"})

    # Changing "Matches" column to Integer
    df['Matches'] = pd.to_numeric( df['Matches'], downcast='integer', errors='coerce')

    return df

In [89]:
# Function to transform champions DataFrame

def champions_list_transform():
    for i in range(len(leagues_prefixs)):
        df = pd.read_csv(f"./RAW/ArabicSoccer/Champions/{leagues_prefixs[i]}_Champions.csv")
        df = df[["Season", "Club", "League"]]
        df.rename(columns={"Club": "Clubs"}, inplace=True)
        df["id"] = df.index + 1
        df = df[["id", "Clubs", "League", "Season"]]
        # The loop is based in the arabic leagues
        # The i == 1 correspond to the champions of SA1, and the i == 2 of SA2L. The delimiters are to capture all champions from each league from the 2007 to 2023
        if i == 2:
            df = df[:15]
            champions_list.append(df)
        else:
            df = df[:16]
            champions_list.append(df)

In [90]:
# Function to transform teams DataFrames

def teams_transform():
    for y in range(len(leagues_prefixs)):
        df = pd.read_csv(f"./RAW/ArabicSoccer/Teams/{leagues_prefixs[y]}_teams_2023.csv")
        df = df[["Club.1", "name", "Squad", "ø age","Foreigners", "ø market value", "League"]]
        
        # Rename columns
        df.rename(columns={"Club.1": "Clubs", "name": "Players", "Squad": "avg_age", "ø age": "Foreigners",
                  "Foreigners": "market_value(€)", "ø market value": "total_market_value(€)"}, inplace=True)

        # I used regex to replace special characters with empty strings, to transform into an integer later
        for i, row in df.iterrows():
            df.iloc[i, 4] = re.sub(pattern='[€mkK]', string=row["market_value(€)"], repl="")
            df.iloc[i, 5] = re.sub(pattern='[€mkK]', string=row["total_market_value(€)"], repl="")

        # Removes the last column, which is an aggregation of data, present in the original table
        df = df[:-1]
        df["id"] = df.index + 1

        # Changing columns dtypes
        df["Foreigners"] = pd.to_numeric(df["Foreigners"], downcast='integer', errors='coerce')
        df['market_value(€)'] = pd.to_numeric(df['market_value(€)'], downcast='float', errors='coerce')
        df['total_market_value(€)'] = pd.to_numeric(df['total_market_value(€)'], downcast='float', errors='coerce')

        # Round columns
        df['market_value(€)'] = df['market_value(€)'].round()
        df['total_market_value(€)'] = df['total_market_value(€)'].round()

        # Change the columns order
        df = df[["id", "Clubs", "Players", "avg_age", "Foreigners","market_value(€)", "total_market_value(€)", "League"]]

        clubs_list.append(df)

Performing the transformation

In [91]:
df_attendances = attendances_tranform()

teams_transform()
df_teams = pd.concat(clubs_list)

champions_list_transform()
df_champions = pd.concat(champions_list)

In [92]:
df_f = pd.read_csv(f"./RAW/ArabicSoccer/Champions/{leagues_prefixs[0]}_Champions.csv")
df_f.head()

,Season,wappen,Club,Club.1,Top coaches,League
0,22/23,NaN,Al-Ittihad Club,Nuno Espírito Santo,NaN,Yelo League
1,21/22,NaN,Al-Hilal SFC,Ramón Díaz,NaN,Yelo League
2,20/21,NaN,Al-Hilal SFC,José Morais,NaN,Yelo League
3,19/20,NaN,Al-Hilal SFC,Razvan Lucescu,NaN,Yelo League
4,18/19,NaN,Al-Nassr FC,Rui Vitória,NaN,Yelo League


In [93]:
df_champions.head()

,id,Clubs,League,Season
0,1,Al-Ittihad Club,Yelo League,22/23
1,2,Al-Hilal SFC,Yelo League,21/22
2,3,Al-Hilal SFC,Yelo League,20/21
3,4,Al-Hilal SFC,Yelo League,19/20
4,5,Al-Nassr FC,Yelo League,18/19


Transform DataFrames in CSV file and send to Trusted Zone.

In [94]:
df_attendances.to_csv(f"./TRUSTED/ArabicSoccer/Attendances/Attendances_SPL_2007_2023.csv", encoding='utf-8', index=False)
df_teams.to_csv(f"./TRUSTED/ArabicSoccer/Teams/Teams_Arabia.csv", encoding='utf-8', index=False)
df_champions.to_csv(f"./TRUSTED/ArabicSoccer/Champions/Champions_Arabia_2007_2023.csv", encoding='utf-8', index=False)